## PIPELINE

### generate expression profile

In [8]:
%%bash
echo "
### Reading Reference
GEN_DIR	../reference/GRCh37_by_chrom/
REF_FILE_NAME	../reference/gencode.v31lift37.annotation.gtf

### Expression
PRO_FILE_NAME	../profiles/count_30M.PRO
NB_MOLECULES	30000000
TSS_MEAN	50
POLYA_SCALE	NaN
POLYA_SHAPE	NaN
" > ./paraFiles/expression.PAR

echo '
#! /bin/bash
#$ -S /bin/bash
#$ -R y
#$ -cwd
#$ -V
#$ -m ea
#$ -M wyynju1993@gmail.com
#$ -j y
#$ -l h_vmem=15G,m_mem_free=15G
#$ -pe smp 4
#$ -N flux-exp
flux-simulator -p ../paraFiles/expression.PAR -x
' > ./qsub_respublica/qsub_flux-exp.sh

### moidfy expression profile column 6 METTL4 expression profile

In [1]:
%%bash
rm profiles/abundance.tsv
ln -s /mnt/isilon/xing_lab/wangy14/project/METTL4/exp/kallisto/293T-WT-1/abundance.tsv profiles/abundance.tsv

In [6]:
count = {}
with open('profiles/abundance.tsv', 'r') as fin:
    fin.readline()
    for line in fin:
        target_id, length, eff_length, est_counts, tpm = line.strip().split('\t')
        target_id = target_id.split('|')[0]
        count[target_id] = int(float(tpm) * 50)

with open('profiles/count_30M.PRO', 'r') as fin, open('profiles/count_modified.PRO', 'w') as fout:
    for line in fin:
        x = line.strip().split('\t')
        tx = x[1]
        if tx in count:
            fout.write('\t'.join(x[:5]) + '\t' + str(count[tx]) + '\n')

In [7]:
%%bash
awk -F '\t' '{sum += $6} END {print sum}' profiles/count_modified.PRO

45756065


### generate libraries

In [9]:
%%bash
echo '
### Reading Reference
GEN_DIR	../reference/GRCh37_by_chrom/
REF_FILE_NAME	../reference/gencode.v31lift37.annotation.gtf
PRO_FILE_NAME	../profiles/count_modified.PRO

### Library Construction
#### fragmentation
# FRAG_SUBSTRATE	RNA
# FRAG_METHOD	UR
# FRAG_UR_ETA	NaN
# FRAG_UR_D0	1
#### Reverse Transcription
# RTRANSCRIPTION	true
# RT_PRIMER	RH
RT_LOSSLESS	YES
# RT_MIN	500
# RT_MAX	5500
#### Filtering & Amplification
FILTERING	true
GC_MEAN	NaN
PCR_PROBABILITY	0.05
LIB_FILE_NAME	../libraries/count_modified.LIB
' > ./paraFiles/lib.PAR

In [10]:
%%bash
echo '
#! /bin/bash
#$ -S /bin/bash
#$ -R y
#$ -cwd
#$ -V
#$ -m ea
#$ -M wyynju1993@gmail.com
#$ -j y
#$ -l h_vmem=15G,m_mem_free=15G
#$ -pe smp 4
#$ -N flux-lib
flux-simulator -p ../paraFiles/lib.PAR -l
' > ./qsub_respublica/qsub_flux-lib.sh

### generate sequences

In [11]:
%%bash
echo '
GEN_DIR	../reference/GRCh37_by_chrom/
REF_FILE_NAME	../reference/gencode.v31lift37.annotation.gtf
LIB_FILE_NAME	../libraries/count_modified.LIB
READ_LENGTH	76
PAIRED_END	true
ERR_FILE	76
FASTA	true
UNIQUE_IDS	NO
READ_NUMBER	20000000
SEQ_FILE_NAME	../fastq/10M.bed
' > paraFiles/seq_10M.PAR

echo '
GEN_DIR	../reference/GRCh37_by_chrom/
REF_FILE_NAME	../reference/gencode.v31lift37.annotation.gtf
LIB_FILE_NAME	../libraries/count_modified.LIB
READ_LENGTH	76
PAIRED_END	true
ERR_FILE	76
FASTA	true
UNIQUE_IDS	NO
READ_NUMBER	40000000
SEQ_FILE_NAME	../fastq/20M.bed
' > paraFiles/seq_20M.PAR

echo '
GEN_DIR	../reference/GRCh37_by_chrom/
REF_FILE_NAME	../reference/gencode.v31lift37.annotation.gtf
LIB_FILE_NAME	../libraries/count_modified.LIB
READ_LENGTH	76
PAIRED_END	true
ERR_FILE	76
FASTA	true
UNIQUE_IDS	NO
READ_NUMBER	60000000
SEQ_FILE_NAME	../fastq/30M.bed
' > paraFiles/seq_30M.PAR

echo '
GEN_DIR	../reference/GRCh37_by_chrom/
REF_FILE_NAME	../reference/gencode.v31lift37.annotation.gtf
LIB_FILE_NAME	../libraries/count_modified.LIB
READ_LENGTH	76
PAIRED_END	true
ERR_FILE	76
FASTA	true
UNIQUE_IDS	NO
READ_NUMBER	80000000
SEQ_FILE_NAME	../fastq/40M.bed
' > paraFiles/seq_40M.PAR

echo '
GEN_DIR	../reference/GRCh37_by_chrom/
REF_FILE_NAME	../reference/gencode.v31lift37.annotation.gtf
LIB_FILE_NAME	../libraries/count_modified.LIB
READ_LENGTH	76
PAIRED_END	true
ERR_FILE	76
FASTA	true
UNIQUE_IDS	NO
READ_NUMBER	100000000
SEQ_FILE_NAME	../fastq/50M.bed
' > paraFiles/seq_50M.PAR

In [12]:
%%bash
echo 'flux-simulator -p ../paraFiles/seq_10M.PAR -s
flux-simulator -p ../paraFiles/seq_20M.PAR -s
flux-simulator -p ../paraFiles/seq_30M.PAR -s
flux-simulator -p ../paraFiles/seq_40M.PAR -s
flux-simulator -p ../paraFiles/seq_50M.PAR -s' > ./qsub_respublica/run_flux-seq.txt

echo '
#! /bin/bash
#$ -S /bin/bash
#$ -R y
#$ -cwd
#$ -V
#$ -m ea
#$ -M wyynju1993@gmail.com
#$ -j y
#$ -l h_vmem=15G,m_mem_free=15G
#$ -pe smp 4
#$ -t 1-5:1
#$ -N flux-seq
`sed -n ${SGE_TASK_ID}p run_flux-seq.txt`
' > ./qsub_respublica/qsub_flux-seq.sh

## Calculate PSI

In [1]:
%%bash
python ./src/calculatePSI.py --gtf ./reference/gencode.v31lift37.annotation.gtf --pro ./profiles/count_modified.PRO --rmats ../rmats/post/fromGTF.MXE.txt,../rmats/post/fromGTF.RI.txt,../rmats/post/fromGTF.A3SS.txt,../rmats/post/fromGTF.A5SS.txt,../rmats/post/fromGTF.SE.txt


start: parsing exp profile
end  : parsing exp profile
start: detecting SE
end  : detecting SE
start: calculating PSI
end  : calculating PSI


## UNDERSTAND parameter

### 30M NB_molecule example

In [2]:
%%bash
echo "
### Reading Reference
GEN_DIR	../reference/GRCh37_by_chrom/
REF_FILE_NAME	../reference/gencode.v31lift37.annotation.gtf

### Expression
NB_MOLECULES	30000000
TSS_MEAN	50
POLYA_SCALE	NaN
POLYA_SHAPE	NaN
PRO_FILE_NAME	../profiles/count_modified.PRO

### Library Construction
#### fragmentation
# FRAG_SUBSTRATE	RNA
# FRAG_METHOD	UR
# FRAG_UR_ETA	NaN
# FRAG_UR_D0	1
#### Reverse Transcription
# RTRANSCRIPTION	true
# RT_PRIMER	RH
RT_LOSSLESS	YES
# RT_MIN	500
# RT_MAX	5500
#### Filtering & Amplification
FILTERING	true
GC_MEAN	NaN
PCR_PROBABILITY	0.05
# SIZE_DISTRIBUTION	cython_rMATS/frag_dist.txt	#Frag_dist.txt is a single-column text file storing the first 10000 fragment length.
LIB_FILE_NAME	../libraries/count_modified.LIB

### sequencing
READ_LENGTH	76
PAIRED_END	true
ERR_FILE	76
FASTA	true
UNIQUE_IDS	NO
READ_NUMBER	60000000
SEQ_FILE_NAME	../fastq/30M.bed
" > paraFiles/example_unmodified.PAR

### example from NBT 
***Sailfish enables alignment-free isoform quantification from RNA-seq reads using lightweight algorithms***
